In [1]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7

In [2]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
from guidance import gen
from evaluate import load
import csv
import nltk.translate.bleu_score as bleu
from rouge import Rouge
from transformers import BertTokenizer, BertModel
import torch
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import warnings
from typing import Any, Dict, Tuple
import re

warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

In [15]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Load Dataset

- id: unique identifier of the example
- source: original model input for paraphrase generation
- hyp1: first alternative paraphrase of the source
- hyp2: second alternative paraphrase of the source
- label: hyp1 or hyp2, based on which of those has been annotated as hallucination
- type: hallucination category assigned.
  - Possible values: addition, named-entity, number, conversion, date, tense, negation, gender, pronoun, antonym, natural

## Trial dataset

This is a small list of examples, provided to help the participants get familiar with the task. Each example contains the following fields:
- id
- source
- hyp1
- hyp2
- label
- type

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [9]:
from datasets import load_dataset
#load the test data for the cross-model evaluation in both English and Swedish
data = load_dataset("Eloquent/HalluciGen-PG", "cross_model_evaluation")

print(data.keys())

data_df_en = data['cross_model_evaluation_english'].to_pandas()
data_df_se= data['cross_model_evaluation_swedish'].to_pandas()



Generating cross_model_evaluation_swedish split:   0%|          | 0/380 [00:00<?, ? examples/s]

Generating cross_model_evaluation_english split:   0%|          | 0/595 [00:00<?, ? examples/s]

dict_keys(['cross_model_evaluation_swedish', 'cross_model_evaluation_english'])


In [10]:
data_df_en

,id,source,hyp1,hyp2
0,0,I would also like to draw attention to Amendme...,the speaker wants to highlight amendments nos...,the speaker mentioned some random amendments ...
1,1,"It is we politicians who are most concerned, b...",Politicians are the most concerned because the...,Politicians are not really worried about the r...
2,2,An important factor here is that the pay and o...,It is crucial that the salary and benefits of ...,One key aspect to consider is that the compens...
3,3,There are four points in relation to which the...,The Committee on Agriculture and Rural Develop...,The Committee on Agriculture and Rural Develop...
4,4,People would think we were mad if we said tomo...,If we were to announce tomorrow that drivers c...,It would be considered normal if we suddenly d...
...,...,...,...,...
590,590,"The Member States are, in other words, being a...",The Member States are being asked to harmonize...,The Member States are being asked to harmonise...
591,591,We struggle with water on a daily basis in the...,"In the Netherlands, water is a daily struggle,...","Water is readily available in the Netherlands,..."
592,592,"On the contrary, things came to a head in the ...","In contrast, the issue escalated during the cr...","Conversely, the situation reached a critical p..."
593,593,It is extremely important that the forthcoming...,It is imperative that the upcoming European Co...,It is crucial that the upcoming European Counc...


In [11]:
data_df_se

,id,source,hyp1,hyp2
0,0,Romney tillkännager Paul Ryan som vicepresiden...,romney utser paul ryan som sin running mate i...,romney offentliggör att paul ryan blir hans m...
1,1,Syriens militärpolischef hoppade av till oppos...,Syriens militärpolischef hoppade av till oppos...,Syriens militärpolischef hoppade av till oppos...
2,2,"Yahoo går med på att köpa Tumblr för 1,1 milja...",Yahoo har för avsikt att sälja Tumblr till en ...,Yahoo har avtalat att förvärva Tumblr för en s...
3,3,"Önationen rapporterade 65 nya fall den 22 maj,...",Taiwans epidemi har inte sett någon ökning av ...,Taiwans epidemi har sett en ökning av 65 nya f...
4,4,Nordkorea säger att en amerikansk medborgare s...,Nordkoreas ledare Kim Jong-un säger att en ame...,Nordkoreas ledare Kim Jong-un säger att en ame...
...,...,...,...,...
375,375,Påven uppmanar till åtgärder mot klimatförändr...,Påven uppmanar till åtgärder mot klimatförändr...,Påven uppmanar till åtgärder mot klimatförändr...
376,376,"Andy Murray förtjänaren adelstitel, säger Davi...","Andy Murray förtjänaren adelstitel, säger Davi...","Andy Murray förtjänaren adelstitel, säger Davi..."
377,377,Beväpnad man dödar 6 i skottlossning mot Wisco...,En ensam gärningsman har skjutit sex personer ...,En beväpnad individ har dödat sex personer i e...
378,378,Ska jag skriva på det här papperet? Ska jag fö...,Om man skriver på det här papperet och förneka...,Det är inte alltid rätt att säga att någon som...


# Load the model

In [8]:
#load model Llama3
from transformers import AutoTokenizer, pipeline
import torch

model = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Prompt Version 1

In [4]:
def detect_hallu(source, hyp1, hyp2):
   # Generate the prompt

  answer_format = {"label": " "}

  user_prompt = f'''
   Given a "src" and two hypotheses "hyp1" and "hyp2" your task is to detect which of the two hypotheses ("label") is not supported by the source.
   Provide the result in the following format: {answer_format}.

   Src: {source}
   hyp1 : {hyp1}
   hyp2 : {hyp2}


   Result:
  '''

  messages = [
    {"role": "system", "content": "You are a hallucination detector. Your task is to detect which hypothesis is a hallucination."},
    {"role": "user", "content": user_prompt},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=150,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
  print("Label: ", outputs[0]["generated_text"][len(prompt):])
  hallu_label = outputs[0]["generated_text"][len(prompt):]

  return hallu_label

In [5]:
def detect_hallu_se(source, hyp1, hyp2):
   # Generate the prompt

    answer_format = {"label": ""}

    user_prompt = f'''
    Givet en ”src” och två hypoteser ”hyp1” och ”hyp2” är din uppgift att upptäcka vilken av de två hypoteserna (”label”) som inte stöds av källan.
    Ge resultatet i följande format: {answer_format}.

    Src: {source}
    hyp1 : {hyp1}
    hyp2 : {hyp2}

    Resultat:
    '''

    messages = [
      {"role": "system", "content": "Du är en hallucinationsdetektor. Din uppgift är att upptäcka vilken hypotes som är en hallucination."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    print("Label: ", outputs[0]["generated_text"][len(prompt):])
    hallu_label = outputs[0]["generated_text"][len(prompt):]


    return hallu_label

In [6]:
import json
import re

def extract_json_data(result_text):

    print(result_text)
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    print("result_text:",result_text)
    return result_text

In [12]:
source = "My mom's food is better than my sister's."
hyp1= "My mom's food is amazing"
hyp2="My sister's food is better than my mom's"
result = detect_hallu(source,hyp1,hyp2)
print()
print(result)
result = extract_json_data(result)
print()

print("result:",result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}

{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}

result: {'label': 'hyp2'}


## Generate dataset

In [13]:
import os
import numpy as np
import pandas as pd

#save results
def save_results(detect_hallu, df, csv_filename):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = detect_hallu(source, hyp1, hyp2)

                print(result)
                extracted_hallu = extract_json_data(result)

                hallu_label = extracted_hallu["label"]


                # Update DataFrame with new values
                chunk.at[index, "prediction"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows

# Generate predictions on the test data

## English

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/cross_model_detection_task_llama3/results_llama3_cross_model_en_v1.csv"

save_results(detect_hallu, data_df_en, csv_filename)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}

Explanation: The source text mentions specific amendments (Nos 27, 29, and 32) and their connection to public inquiries or consultation in connection with trial releases. This supports hypothesis 1, which specifically mentions the connection between the amendments and public involvement in trials. Hypothesis 2, on the other hand, is not supported by the source text, as it does not mention any specific connection between the amendments and trials or public participation.
{'label': 'hyp2'}

Explanation: The source text mentions specific amendments (Nos 27, 29, and 32) and their connection to public inquiries or consultation in connection with trial releases. This supports hypothesis 1, which specifically mentions the connection between the amendments and public involvement in trials. Hypothesis 2, on the other hand, is not supported by the source text, as it does not mention any specific connection between the amendments and trials or public participation.
{'la

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}

Explanation: The source text mentions that "it is we politicians who are most concerned" and "we understand the link between energy consumption and CO2 emissions", which supports hypothesis 1. However, hypothesis 2 contradicts the source text by stating that politicians are not worried about the relationship between energy consumption and CO2 emissions, which is a hallucination.
{'label': 'hyp2'}

Explanation: The source text mentions that "it is we politicians who are most concerned" and "we understand the link between energy consumption and CO2 emissions", which supports hypothesis 1. However, hypothesis 2 contradicts the source text by stating that politicians are not worried about the relationship between energy consumption and CO2 emissions, which is a hallucination.
{'label': 'hyp2'}

Explanation: The source text mentions that "it is we politicians who are most concerned" and "we understand the link between energy consumption and CO2 emissions", which s

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}
{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}

The source text does not mention "gathering" which is present in hyp2. The other two hypotheses are supported by the source text.
{'label': 'hyp2'}

The source text does not mention "gathering" which is present in hyp2. The other two hypotheses are supported by the source text.
{'label': 'hyp2'}

The source text does not mention "gathering" which is present in hyp2. The other two hypotheses are supported by the source text.
result_text: {'label': 'hyp2'}
Label:  {'label': 'hyp2'}
{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}


[]

In [ ]:
# save a csv file in the task format

import pandas as pd

# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/cross_model_detection_task_llama3/results_llama3_cross_model_en_v1.csv")

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/final_lama3_cross_model_en_v1.csv', index=False)

## Swedish

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/cross_model_detection_task_llama3/results_llama3_cross_model_se_v1.csv"

save_results(detect_hallu_se, data_df_se, csv_filename)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}
{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}
{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp2'}
{'label': 'hyp2'}
{'label': 'hyp2'}
result_text: {'label': 'hyp2'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Label:  {'label': 'hyp1'}
{'label': 'hyp1'}
{'label': 'hyp1'}
result_text: {'label': 'hyp1'}
Label:  {'label': 'hyp2'}

The source text "Egypten fryser Muslimska brödraskapets tillgångar" implies that Egypt has frozen the assets of the Muslim Brotherhood, but it does not provide any information about the impact on the demand for Islamist terrorism. Therefore, both hypotheses are not supported by the source text. However, since the source text only mentions the freezing of assets, it is more likely that the correct hypothesis is the one that does not make a claim about the impact on terrorism demand.
{'label': 'hyp2'}

The source text "Egypten fryser Muslimska brödraskapets tillgångar" implies that Egypt has frozen the assets of the Muslim Brotherhood, but it does not provide any information about the impact on the demand for Islamist terrorism. Therefore, both hypotheses are not supported by the source text. However, since the source text only mentions the freezing of assets, it is mor

[351]

In [ ]:
# save a csv file in the task format

import pandas as pd

# Load the CSV file
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/cross_model_detection_task_llama3/results_llama3_cross_model_se_v1.csv")

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/cross_model_detection_task/final_lama3_cross_model_se_v1.csv', index=False)